# Pycoornet Example

In [1]:
# Install pycoornet
!pip install pycoornet -q

## Import libraries

In [2]:
from pycoornet.crowdtangle import CrowdTangle
from pycoornet.shared import Shared
from pycoornet.statistics import Statistics
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import logging

## Get Links

In [3]:
# Read data links
links_df = pd.read_csv('sample_source_links.csv')

In [4]:
links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url_rid    2 non-null      object
 1   clean_url  2 non-null      object
 2   date       2 non-null      object
dtypes: object(3)
memory usage: 176.0+ bytes


In [5]:
links_df.rename(columns={'first_post_time': 'date'}, inplace=True)

## Get Crowdtangle information

In [6]:
# start logging info
logging.basicConfig(level=logging.INFO, filename="pycoornet.log")

In [7]:
# init Crowtangle with api key
crowd_tangle = CrowdTangle("xxxxxxxxxxxx")

In [8]:
#Get links from crowdtangle
ct_df = crowd_tangle.get_shares(urls=links_df, url_column='clean_url', date_column='date',clean_urls=True, platforms='facebook', sleep_time=4)

100%|██████████| 2/2 [00:14<00:00,  7.49s/it]


In [9]:
shared = Shared()
crowtangle_shares_df, shares_graph, q = shared.coord_shares(ct_df, clean_urls=True)

100%|██████████| 2/2 [00:00<00:00, 28.94it/s]


## Build Gephi File

In [10]:
for node in shares_graph.nodes(data=True):
    node[1]['label']=node[1]['account_name']
nx.write_gexf(shares_graph, "out/shares.gexf")

In [11]:
coord_graph = shares_graph

In [12]:
coord_graph.remove_edges_from(nx.selfloop_edges(coord_graph))

In [13]:
layout = nx.spring_layout(coord_graph,iterations=50)

In [14]:
account_urls = list(crowtangle_shares_df['account_url'].unique())
urls = list(crowtangle_shares_df['link'].unique())

### Plot Network

In [15]:
plt.figure(figsize=(15, 15))
#url_size = [coord_graph.degree(url)*80 for url in urls]
nx.draw_networkx_nodes(coord_graph, 
                       layout, 
                       nodelist=urls, 
                       #node_size=url_size,
                       node_color='lightblue')
nx.draw_networkx_nodes(coord_graph, layout, nodelist=urls, node_color='#cccccc', node_size=100)
# Draw Popular links
popular_account_url = [account_url for account_url in account_urls if coord_graph.degree(account_url) > 3]
nx.draw_networkx_nodes(coord_graph, layout, nodelist=popular_account_url, node_color='orange', node_size=100)
nx.draw_networkx_edges(coord_graph, layout, width=1, edge_color="#cccccc")
plt.show()

NetworkXError: Node 'https://www.telesurenglish.net/news/UNGA-Overwhelmingly-Votes-Against-Cuba-Blockade-All-US-Amendments-Defeated-20181101-0027.html' has no position.

## Get General Statistical Information

In [16]:
componet_summary_df = Statistics.component_summary(crowtangle_shares_df, shares_graph)

In [17]:
componet_summary_df

,component,entities,avg_subscriber_count,coor_share_ratio_avg,coor_score_avg,page_admin_top_country,facebook_page,facebook_group,facebook_profile,unique_full_domain,unique_parent_domain,gini_full_domain,gini_parent_domain,top_full_domain,top_parent_domain
0,1,150,30428.773333,0.5,1.000268,nan,85,65,0.0,1,1,0.0,0.0,[www.telesurenglish.net],[telesurenglish.net]


In [18]:
top_urls_df = Statistics.get_top_coord_urls(crowtangle_shares_df, shares_graph)

In [19]:
top_urls_df

,expanded,statistics_actual_likeCount,statistics_actual_shareCount,statistics_actual_commentCount,statistics_actual_loveCount,statistics_actual_wowCount,statistics_actual_hahaCount,statistics_actual_sadCount,statistics_actual_angryCount,engagement,count,account_url,coor_account_url,account_name,coor_account_name,components
0,https://www.telesurenglish.net/news/Bolivia-Ro...,13101,9151,906,2898,520,36,35,17,26664,108,"[https://www.facebook.com/1031686746989127, ht...","[https://www.facebook.com/1031686746989127, ht...","[AFROMEXICO, Adriana Avendaño Christie, Age of...","[AFROMEXICO, Adriana Avendaño Christie, Age of...",[1]
1,https://www.telesurenglish.net/news/UNGA-Overw...,3916,4143,257,857,142,34,12,34,9395,50,"[https://www.facebook.com/1147524838680378, ht...","[https://www.facebook.com/1147524838680378, ht...","[#UniteTheTribes, AZANIAN TRUE POLITICAL HISTO...","[#UniteTheTribes, Asuntos Cubanos, Australia C...",[1]
